# Тема “Предобработка текста с помощью Python”

Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1. Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

1. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

1. Изменим регистр твитов на нижний с помощью .lower().

1. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

1. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

1. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

1. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

1. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

1. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

1. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

1. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

1. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

1. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

1. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

1. Сохраним результат предобработки в pickle-файл.

In [1]:
apostrophe_dict = {
    "ain't": "am not / are not",
    "aren't": "are not / am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is",
    "i'd": "I had / I would",
    "i'd've": "I would have",
    "i'll": "I shall / I will",
    "i'll've": "I shall have / I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"
}

short_word_dict = {
    "121": "one to one",
    "a/s/l": "age, sex, location",
    "adn": "any day now",
    "afaik": "as far as I know",
    "afk": "away from keyboard",
    "aight": "alright",
    "alol": "actually laughing out loud",
    "b4": "before",
    "b4n": "bye for now",
    "bak": "back at the keyboard",
    "bf": "boyfriend",
    "bff": "best friends forever",
    "bfn": "bye for now",
    "bg": "big grin",
    "bta": "but then again",
    "btw": "by the way",
    "cid": "crying in disgrace",
    "cnp": "continued in my next post",
    "cp": "chat post",
    "cu": "see you",
    "cul": "see you later",
    "cul8r": "see you later",
    "cya": "bye",
    "cyo": "see you online",
    "dbau": "doing business as usual",
    "fud": "fear, uncertainty, and doubt",
    "fwiw": "for what it's worth",
    "fyi": "for your information",
    "g": "grin",
    "g2g": "got to go",
    "ga": "go ahead",
    "gal": "get a life",
    "gf": "girlfriend",
    "gfn": "gone for now",
    "gmbo": "giggling my butt off",
    "gmta": "great minds think alike",
    "h8": "hate",
    "hagn": "have a good night",
    "hdop": "help delete online predators",
    "hhis": "hanging head in shame",
    "iac": "in any case",
    "ianal": "I am not a lawyer",
    "ic": "I see",
    "idk": "I don't know",
    "imao": "in my arrogant opinion",
    "imnsho": "in my not so humble opinion",
    "imo": "in my opinion",
    "iow": "in other words",
    "ipn": "I’m posting naked",
    "irl": "in real life",
    "jk": "just kidding",
    "l8r": "later",
    "ld": "later, dude",
    "ldr": "long distance relationship",
    "llta": "lots and lots of thunderous applause",
    "lmao": "laugh my ass off",
    "lmirl": "let's meet in real life",
    "lol": "laugh out loud",
    "ltr": "longterm relationship",
    "lulab": "love you like a brother",
    "lulas": "love you like a sister",
    "luv": "love",
    "m/f": "male or female",
    "m8": "mate",
    "milf": "mother I would like to fuck",
    "oll": "online love",
    "omg": "oh my god",
    "otoh": "on the other hand",
    "pir": "parent in room",
    "ppl": "people",
    "r": "are",
    "rofl": "roll on the floor laughing",
    "rpg": "role playing games",
    "ru": "are you",
    "shid": "slaps head in disgust",
    "somy": "sick of me yet",
    "sot": "short of time",
    "thanx": "thanks",
    "thx": "thanks",
    "ttyl": "talk to you later",
    "u": "you",
    "ur": "you are",
    "uw": "you’re welcome",
    "wb": "welcome back",
    "wfm": "works for me",
    "wibni": "wouldn't it be nice if",
    "wtf": "what the fuck",
    "wtg": "way to go",
    "wtgp": "want to go private",
    "ym": "young man",
    "gr8": "great"
}


emoticon_dict = {
    ":\)": "happy",
    ":‑\)": "happy",
    ":-\]": "happy",
    ":-3": "happy",
    ":->": "happy",
    "8-\)": "happy",
    ":-\}": "happy",
    ":o\)": "happy",
    ":c\)": "happy",
    ":\^\)": "happy",
    "=\]": "happy",
    "=\)": "happy",
    "<3": "happy",
    ">:\[": "sad",
    ">:\(": "sad",
    ":-\(": "sad",
    ":\(": "sad",
    ":c": "sad",
    ":<": "sad",
    ":\[": "sad",
    ":\{": "sad",
    ":-c": "sad",
    ":-< ": "sad",
    ":-\[": "sad",
    ":-\|\|": "sad"
}

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [3]:
from html.parser import HTMLParser

In [4]:
train_df = pd.read_csv('train_tweets.tar.xz')
train_df.head()

,train_tweets.csv,label,tweet
0,1.0,0.0,@user when a father is dysfunctional and is s...
1,2.0,0.0,@user @user thanks for #lyft credit i can't us...
2,3.0,0.0,bihday your majesty
3,4.0,0.0,#model i love u take with u all the time in ...
4,5.0,0.0,factsguide: society now #motivation


In [5]:
test_df = pd.read_csv('test_tweets.tar.xz')
test_df.head()

,test_tweets.csv,tweet
0,31963.0,#studiolife #aislife #requires #passion #dedic...
1,31964.0,@user #white #supremacists want everyone to s...
2,31965.0,safe ways to heal your #acne!! #altwaystohe...
3,31966.0,is the hp and the cursed child book up for res...
4,31967.0,"3rd #bihday to my amazing, hilarious #nephew..."


In [6]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,train_tweets.csv,label,tweet,test_tweets.csv
0,1.0,0.0,@user when a father is dysfunctional and is s...,NaN
1,2.0,0.0,@user @user thanks for #lyft credit i can't us...,NaN
2,3.0,0.0,bihday your majesty,NaN
3,4.0,0.0,#model i love u take with u all the time in ...,NaN
4,5.0,0.0,factsguide: society now #motivation,NaN


In [7]:
def preprocess(combine_df):
    # 1. Заменим html-сущности (к примеру: \&lt; \&gt; \&amp;). "&lt;" заменим на “<” и "\&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape()
    
    html_parser = HTMLParser()
    combine_df = combine_df[combine_df.tweet.notna()]    
    combine_df.tweet = combine_df.tweet.apply(lambda tweet: html_parser.unescape(tweet))
    
    # 2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
    # - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
    # - для для замены @user на пробел, необходимо использовать re.sub()
    # при применении функции необходимо использовать np.vectorize(function)
    
    combine_df = combine_df.replace(to_replace='@\w+', value='', regex=True)
    
    # 3. Изменим регистр твитов на нижний с помощью .lower()
    
    combine_df.tweet = combine_df.tweet.str.lower()
    
    # 4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).
    
    combine_df.tweet = combine_df.tweet.replace(apostrophe_dict, regex=True)
    
    # 5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.
    
    combine_df.tweet = combine_df.tweet.replace(short_word_dict, regex=True)
    
    # 6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.
    
    combine_df.tweet = combine_df.tweet.replace(emoticon_dict, regex=True)
    
    # 7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'
    
    combine_df.tweet = combine_df.tweet.replace(to_replace='[^\w\s]', value=' ', regex=True)
    
    # 8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'
    
    combine_df.tweet = combine_df.tweet.replace(to_replace='[^a-zA-Z0-9]', value=' ', regex=True)
    
    # 9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'
    
    combine_df.tweet = combine_df.tweet.replace(to_replace='[^a-zA-Z]', value=' ', regex=True)
    
    # 10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])
    
    combine_df.tweet = combine_df.tweet.apply(lambda tweet: ' '.join([w for w in tweet.split() if len(w)>1]))
    
    # 11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.
    
    combine_df['tweet_token'] = combine_df.tweet.apply(lambda tweet: nltk.word_tokenize(tweet))
    
    # 12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.
    
    stop_words = set(stopwords.words("english"))
    
    combine_df['tweet_token_filtered'] = combine_df.tweet_token.apply(lambda words: [word for word in words if not word in stop_words])
    
    # 13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.
    
    stemmer = PorterStemmer()
    
    combine_df['tweet_stemmed'] = combine_df.tweet_token_filtered.apply(lambda row: list(map(stemmer.stem, row)))
    
    # 14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.
    
    lemmatizer = WordNetLemmatizer()
    
    combine_df['tweet_lemmatized'] = combine_df.tweet_token_filtered.apply(lambda row: list(map(lemmatizer.lemmatize, row)))
    return combine_df

In [8]:
result = preprocess(combine_df.copy())

/home/dmitry/.virtualenvs/geekbrains_nlp/lib/python3.8/site-packages/pandas/core/generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


15. Сохраним результат предобработки в pickle-файл.

In [9]:
result

,train_tweets.csv,label,tweet,test_tweets.csv,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1.0,0.0,when fatheare is dysfyounctional and is so sel...,NaN,"[when, fatheare, is, dysfyounctional, and, is,...","[fatheare, dysfyounctional, selfish, dareagare...","[fathear, dysfyounct, selfish, dareagarein, ki...","[fatheare, dysfyounctional, selfish, dareagare..."
1,2.0,0.0,thanks foare lyft careedit cannot youse cayous...,NaN,"[thanks, foare, lyft, careedit, can, not, yous...","[thanks, foare, lyft, careedit, youse, cayouse...","[thank, foar, lyft, careedit, yous, cayous, of...","[thanks, foare, lyft, careedit, youse, cayouse..."
2,3.0,0.0,bihday yoyouare majesty,NaN,"[bihday, yoyouare, majesty]","[bihday, yoyouare, majesty]","[bihday, yoyouar, majesti]","[bihday, yoyouare, majesty]"
3,4.0,0.0,model love you take with you all the time in y...,NaN,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, youare]","[model, love, take, time, youar]","[model, love, take, time, youare]"
4,5.0,0.0,factsgrinyouide society now motivation,NaN,"[factsgrinyouide, society, now, motivation]","[factsgrinyouide, society, motivation]","[factsgrinyouid, societi, motiv]","[factsgrinyouide, society, motivation]"
...,...,...,...,...,...,...,...,...
49155,NaN,NaN,thoyougareinht factoarey left areigareinht pol...,49155.0,"[thoyougareinht, factoarey, left, areigareinht...","[thoyougareinht, factoarey, left, areigareinht...","[thoyougareinht, factoarey, left, areigareinht...","[thoyougareinht, factoarey, left, areigareinht..."
49156,NaN,NaN,feelingarein like mearemaid haiareflip neveare...,49156.0,"[feelingarein, like, mearemaid, haiareflip, ne...","[feelingarein, like, mearemaid, haiareflip, ne...","[feelingarein, like, mearemaid, haiareflip, ne...","[feelingarein, like, mearemaid, haiareflip, ne..."
49157,NaN,NaN,hillaarey campaigareinned today in ohio oh my ...,49157.0,"[hillaarey, campaigareinned, today, in, ohio, ...","[hillaarey, campaigareinned, today, ohio, oh, ...","[hillaarey, campaigarein, today, ohio, oh, god...","[hillaarey, campaigareinned, today, ohio, oh, ..."
49158,NaN,NaN,happy at woarek confeareence areigareinht mind...,49158.0,"[happy, at, woarek, confeareence, areigareinht...","[happy, woarek, confeareence, areigareinht, mi...","[happi, woarek, confear, areigareinht, mindset...","[happy, woarek, confeareence, areigareinht, mi..."


In [10]:
result.to_pickle('combine_df.pkl')